## Day 24: Blizzard Basin

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day/puzzle24.ipynb)

### Part 1

In [41]:
#!value --name sampleRaw 
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#

In [42]:
#load "../common/common.fsx"
open System.Collections.Generic

type Facing = Point

let down : Facing = Point (1,0)
let up : Facing = Point (-1,0)
let right : Facing = Point (0,1)
let left : Facing = Point (0,-1)


type Blizzard = Facing

type Blizzards = { Map: Blizzard array [,] }
with member this.Entrance = Point(-1, 0)
     member this.Exit = down + Point(Array2D.length1 this.Map - 1, Array2D.length2 this.Map - 1)


In [43]:
let parse s = 
    let m = 
        (Pattern1.read (fun row -> row[1 .. ^1]) s)[1 .. ^1]
        |> array2D
        |> Array2D.map (fun c -> 
            match c with 
            | '>' -> [| right |]
            | 'v' -> [| down |]
            | '<'-> [| left |]
            | '^' -> [| up |]
            | _ -> Array.empty
        )
    { Map = m; }

In [44]:
#!share sampleRaw --from value

let sampleBlizzards = parse sampleRaw

To model wind we make use of the previously defined Euclidean remainder operator `%%` and just build a new dictionary.

In [45]:
let wind' blizzards = 
    
    let X = Array2D.length1 blizzards.Map
    let Y = Array2D.length2 blizzards.Map

    let at (Point(i,j)) = 
        blizzards.Map[i %% X, j %% Y]

    let newMap = 
        blizzards.Map
        |> Array2D.mapi (fun i j b -> 
            let point = Point(i,j)
            [|
                yield! at (point - down) |> Seq.where ((=) down)
                yield! at (point - up) |> Seq.where ((=) up)
                yield! at (point - left) |> Seq.where ((=) left)
                yield! at (point - right) |> Seq.where ((=) right)
            |]
        )
    { Map = newMap }

However, considering the blizzards state at some moment of time, it is worth noting that the state is going to repeat every $\mathrm{LCM} \left(width, height \right)$ steps. The code below relies on funcions `memoize`, `lcm` from the `common.fsx` script.

A memoized function is available via the generator function `wind()` to make the cache controllable.

In [46]:
type State = { Blizzards : Blizzards; Position : Point; Step : int }

let windF() = 
    memoize 
        (fun state -> wind' state.Blizzards) 
        (fun state -> state.Step % lcm (Array2D.length1 state.Blizzards.Map) (Array2D.length2 state.Blizzards.Map))


This notebook relies on a breadth-first search algorithm implementation from the `bfs.fsx` script file. 

In this problem we have a graph of decicions taken during each minute, including waiting, if possible. The graph is traversed in search of the goal.

In [47]:
#load "../common/bfs.fsx"

open Bfs
open Bfs.Custom

let goalTarget : Target<State> =
    fun state -> 
        state.Position = state.Blizzards.Exit

let adjF() : Adjacency<State> =
    let wind = windF()
    fun state ->
        [
            let afterWind = wind state
            
            let options =
                [ down; up; left; right]
                |> List.map ((+)state.Position)
                |> List.filter (fun p -> p = afterWind.Exit
                                        || p = afterWind.Entrance
                                        || afterWind.Map |> Array2D.tryAtPoint p = Some [||]
                                )
                |> List.map(fun p -> { Position = p; Blizzards = afterWind; Step = state.Step + 1})
            
            yield! options

            let canWait = 
                state.Position = afterWind.Exit
                || state.Position = afterWind.Entrance
                || afterWind.Map |> Array2D.atPoint state.Position = [||]
            if canWait
            then
                yield {state with Blizzards = afterWind; Step = state.Step + 1}
        ]

let settings =
    { VisitedKey = fun state -> state.Position, state.Step % lcm (Array2D.length1 state.Blizzards.Map) (Array2D.length2 state.Blizzards.Map)}

let findFoundPath adj target start = 
    let (Found(path)) = findPath settings {Adjacency = adj} start target
    path

In [48]:
let sampleStart = {
    Position = sampleBlizzards.Entrance
    Blizzards = sampleBlizzards
    Step = 0
}

let private sampleResult = 
    findFoundPath (adjF()) goalTarget sampleStart 
    |> List.head
    
sampleResult.Step

18

For the actual input:

In [49]:
#!value --name actualRaw --from-file ./data_actual.txt

In [50]:
#!share actualRaw --from value

let actualBlizzards = parse actualRaw

let actualStart = {
    Position = actualBlizzards.Entrance
    Blizzards = actualBlizzards
    Step = 0
}

In [57]:
#!time 

let private actualResult = 
    findFoundPath (adjF()) goalTarget actualStart
    |> List.head
actualResult.Step

225

Wall time: 1198.5074ms

### Part 2

For part 2, the trips are calculated one by one with intermediate results being piped from search to search. 

The `adjF` function is called once to reuse memoization caches.

In [52]:
let startTarget : Target<State> =
    fun state -> 
        state.Position = state.Blizzards.Entrance

let private adj = adjF()
let private sampleResult = 
    sampleStart
    |> findFoundPath adj goalTarget
    |> List.head
    |> findFoundPath adj startTarget
    |> List.head
    |> findFoundPath adj goalTarget
    |> List.head

sampleResult.Step

54

For the actual input:

In [56]:
#!time

let private adj = adjF()
let private actualResult = 
    actualStart
    |> findFoundPath adj goalTarget
    |> List.head
    |> findFoundPath adj startTarget
    |> List.head
    |> findFoundPath adj goalTarget
    |> List.head

actualResult.Step

711

Wall time: 3750.9903ms